<a>
    <img src="https://ladata-ufs.github.io/Assets/LOGO-EXTENSO-CLARA.png" width=175>
</a>

# **Processamento de Linguagem Natural: das abordagens clássicas aos LLMs**  
> Túlio Gois; Ana Laylla Rocha; Daivik Celeste; David Henrique Ferraz; Letícia Santos.

---
## **Dia 3: Aplicações do PLN**

### **Dependências**

In [ ]:
#instalações
!pip install --quiet -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 57.3 MB/s eta 0:00:00


In [ ]:
#imports
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from transformers import pipeline
from collections import defaultdict
import re

---
## **Recuperação de Informação**

Encontrar material relevante a partir de uma consulta de um usuário.

Surge historicamente na biblioteconomia com a necessidade de organizar grandes acervos e ganha força a partir da década de 1960 com a informatização da busca bibliográfica.

#### **Arquitetura**
Etapas offline:
- Ocorre antes do usuário realizar uma busca;
- Pré-processamento dos documentos: limpeza e normalização;
- Indexação: Organiza os textos em uma estrutura de dados para permitir a busca mais eficiente.

Etapas Online:
- Ocorre quando o usuário solicita uma busca;
- Pré-processamento da consulta: aplica o mesmo pré-processamento dos textos ao material de consulta;
- Busca no índice + Ranqueamento: Encontra e ordena os documentos mais adequados, retornando uma lista ordenada.


#### **Modelos Clássicos**
Booleano:
- Modelo mais antigo e simples;
- Baseado na teoria de conjuntos e álgebra booleana;
- Utiliza os operadores AND, OR, NOT;
- Limitação: Não produz um ranking de documentos relevantes, ou seja, o documento **deve** satisfazer a expressão;
- Apesar de ser um modelo antigo ainda é utilizado em catálogos atuais, como busca de patentes.

Vetorial:
- Já é possível organizar os documentos por grau de relevância;
- Quanto mais vezes o termo de consulta estiver contido no documento maior será sua relevência para a pesquisa;
- Utiliza técnicas como o TF-IDF para calcular os pesos;
- Documentos e consutlas são representados como vetores;
- Similaridade é calculada tipicamente com Cosseno.

Probabilísticos:
- Ordena os documentos de acordo com sua probabilidade de serem relevantes para a consulta (Probability Ranking Principle);
- Estima-se a probabilidade a partir da estatística dos termos em documentos relevantes vs não relevantes.

#### **Avaliação de Qualidade**
Paradigma Cranfield:
- Deve-se possuir uma coleção de teste contendo conjunto de documentos, consultas e o julgamento de relevância;
- Utiliza-se métricas, como Precisão e revocação, para comparar e gerar indicadores se os dados obtidos na consulta correspondem a coleção de teste;

#### **Modificação de consultas**
Com intuito de melhorar os resultados obtidos podemos:
- Expandir a consulta: Adicionar sinônimos e/ou termos morfologicamente ligados;
- Risco: introduzir ruído e desviar o foco semântico da consulta;

In [ ]:
documentos = [
    "A recuperação de informação busca encontrar documentos relevantes em grandes coleções de textos.",
    "Sistemas de busca na web costumam usar o modelo vetorial com pesos TF-IDF para ranquear documentos.",
    "O modelo booleano recupera documentos que satisfazem expressões lógicas com operadores AND e OR.",
    "O pré-processamento inclui tokenização, remoção de stopwords e normalização de termos antes da indexação."
]

vectorizer = TfidfVectorizer(
    lowercase=True,
    stop_words=None,
)
X_docs = vectorizer.fit_transform(documentos)

print("Tamanho do vocabulário (bag-of-words):", len(vectorizer.vocabulary_))


def busca_vetorial(consulta, top_k=3):
    X_query = vectorizer.transform([consulta])
    sims = cosine_similarity(X_query, X_docs)[0]
    ranking = np.argsort(sims)[::-1]
    print(f"\n[Modelo vetorial] Consulta: {consulta!r}")
    for pos, idx in enumerate(ranking[:top_k], start=1):
        print(f"{pos}. Doc {idx} | score = {sims[idx]:.3f}")
        print("   ", documentos[idx])


def busca_booleana_and(consulta):
    analyzer = vectorizer.build_analyzer()
    termos = analyzer(consulta)
    termos = [t for t in termos if t in vectorizer.vocabulary_]

    if not termos:
        print("\n[Booleano AND] Nenhum termo da consulta está no vocabulário.")
        return

    conjuntos_docs = []
    for termo in termos:
        j = vectorizer.vocabulary_[termo]
        docs_com_termo = set(np.where(X_docs[:, j].toarray().ravel() > 0)[0])
        conjuntos_docs.append(docs_com_termo)

    docs_and = set.intersection(*conjuntos_docs) if conjuntos_docs else set()

    print(f"\n[Modelo booleano AND] Consulta: {consulta!r}")
    print("Termos considerados:", termos)
    if not docs_and:
        print("Nenhum documento contém TODOS os termos.")
    else:
        print("Documentos recuperados (contêm todos os termos):", docs_and)
        for idx in sorted(docs_and):
            print(f"- Doc {idx}: {documentos[idx]}")


busca_vetorial("modelo vetorial para recuperação de documentos relevantes")
busca_booleana_and("modelo documentos")


Tamanho do vocabulário (bag-of-words): 44

[Modelo vetorial] Consulta: 'modelo vetorial para recuperação de documentos relevantes'
1. Doc 0 | score = 0.423
    A recuperação de informação busca encontrar documentos relevantes em grandes coleções de textos.
2. Doc 1 | score = 0.398
    Sistemas de busca na web costumam usar o modelo vetorial com pesos TF-IDF para ranquear documentos.
3. Doc 2 | score = 0.135
    O modelo booleano recupera documentos que satisfazem expressões lógicas com operadores AND e OR.

[Modelo booleano AND] Consulta: 'modelo documentos'
Termos considerados: ['modelo', 'documentos']
Documentos recuperados (contêm todos os termos): {np.int64(1), np.int64(2)}
- Doc 1: Sistemas de busca na web costumam usar o modelo vetorial com pesos TF-IDF para ranquear documentos.
- Doc 2: O modelo booleano recupera documentos que satisfazem expressões lógicas com operadores AND e OR.


---
## **Extração de Informação**
Transformar dados textuais não estruturados (texto livre) em informação estruturada (tabelas, banco de dados, comnhecimento organizado)

Primeiros trabalhos surgiu nos anos de 1970 com uso de gramáticas formais e parsers sintáticos para estruturar informação em domínios como prontuários médicos e textos jornalísticos.

#### **Conceituação formal: Relação e Entidade**
Veridador: Fato que torna verdadeira uma proposição
- Salvador pertence à Bahia.

Relação: Tipo abstrato de proposição
- Percentece(Cidade, UF).

Relacionamentos: Instâncias concretas da proposição
- Pertence(Salvador, Bahia).

#### **Extração de Informação Tradicional**
Trabalham com um escopo pré-definido
- Conjunto fechado de entidades e tipos de relações de interesse;
- Portanto, o sistema não irá extrair fatos fora desse escopo.

Exemplos de relações comuns:
- location-of(algo, local): Um aluno pode ser encontrado na escola → location-of(aluno, escola)
- is-a(subclasse, superclasse): Salvador é uma cidade → is-a(Salvador, cidade)
- part-of(parte, todo): Roda é um componente de um carro → part-of(roda, carro)

#### **Reconhecimento de Entidades Nomeadas (REN)**
Identificar e classificar expressões linguísticas que se referem a entidades específicas, como nomes próprios, expressões temporais, espécies biológicas, etc.



#### **Extração de Relações (ER)**
Identificar relacionamentos entre entidades dentro de um escopo/domínio bem definido.

#### **Extração de Informação Aberta (EIA/Open IE)**
Busca extrair informações estruturadas de textos sem pré-definir:
- O domínio de aplicação;
- Relações e tipos de entidade de interesse;
- Generaliza a tarefa da Extração de Relações (ER);
- Tenta extrair todas as relações que apareçam no texto.

In [ ]:
pipe = pipeline(
    "token-classification",
    model="jtlicardo/bpmn-information-extraction",
    aggregation_strategy="simple",
)

texto = """
First, the customer sends a purchase request by e-mail.
Then the sales agent checks the stock availability.
If the product is available, the system creates the order
and sends a confirmation to the customer.
Otherwise, the agent informs the customer that the product is out of stock.
"""

resultados = pipe(texto)

print("=== Saída bruta do modelo (spans e rótulos) ===")
for ent in resultados:
    print(f"{ent['word']:35} | {ent['entity_group']:12} | score={ent['score']:.2f}")

agrupado = defaultdict(list)
for ent in resultados:
    agrupado[ent["entity_group"]].append(ent["word"])

print("\n=== Informações extraídas por tipo ===")
for label, spans in agrupado.items():
    print(f"{label:12} -> {', '.join(spans)}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/431M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cpu


=== Saída bruta do modelo (spans e rótulos) ===
the customer                        | AGENT        | score=0.99
sends a purchase request            | TASK         | score=0.99
by e - mail                         | TASK_INFO    | score=0.49
the sales agent                     | AGENT        | score=0.99
checks the stock availability       | TASK         | score=0.99
If the product is available         | CONDITION    | score=0.99
the system                          | AGENT        | score=0.99
creates the order                   | TASK         | score=1.00
sends a confirmation                | TASK         | score=1.00
to the customer                     | TASK_INFO    | score=0.97
Otherwise                           | CONDITION    | score=0.45
the agent                           | AGENT        | score=0.99
informs the customer                | TASK         | score=0.99
that the                            | TASK_INFO    | score=0.65
product                             | TASK         | sco

---
## **Tradução Automática**
Mapear textos de uma língua-fonte para uma língua-alvo, buscando preservar o significado.

#### **Principais abordagens de Tradução Automática**

Tradução Direta:
- Faz apenas um mapeamento direto de palavras da língua-fonte para a língua-alvo, usando um léxico bilíngue, sem análise sintática ou semântica;
- Não lida com estrutura da língua, ordem de palavras, expressões multi-palavra.

Baseada em regras:
- Utiliza regras linguísticas manuais (morfologia, sintaxe, semântica) + léxicos bilíngues de fonte e alvo.
- Exige muito conhecimento de ambas as línguas, regras difíceis de manter e atualizar, e necessidade de novos conjuntos de regras para cada par de línguas.

Por Interlíngua:
- Traduzir primeiro para uma representação semântico-conceitual independente de língua (interlíngua), e depois dessa representação para qualquer língua-alvo.
- Dificuldade real de criar uma representação realmente universal; cada grupo puxa a interlíngua para suas necessidades, o que limita a adoção ampla.

Baseada em exemplos:
- Traduz "por analogia": aprende a partir de exemplos de sentenças e combina trechos já vistos;
- Mescla métodos estatísticos e neurais.

Tradução Automática Estatística:
- Usa modelos probabilísticos treinados em grandes corpora para aprender como mapear frases de uma língua para outra.

Tradução Automática Neural:
- Abordagem dominante hoje em dia;
- Uma rede neural é treinada para maximizar a probabilidade de uma sentença-alvo dada uma sentença-fonte;
- Necessita de muitos dados de treinamento de boa qualidade, risco de memorizar vieses presentes nos dados e de produzir traduções fluentes porém incorretas.

In [ ]:
translator = pipeline(
    task="translation",
    model="facebook/m2m100_418M",
    tokenizer="facebook/m2m100_418M",
    src_lang="hi",
    tgt_lang="pt"
)

hi_text = "जीवन एक चॉकलेट बॉक्स की तरह है।"

resultado = translator(hi_text)
print("Texto original (hi):", hi_text)
print("Tradução (pt):", resultado[0]["translation_text"])



Device set to use cpu


Texto original (hi): जीवन एक चॉकलेट बॉक्स की तरह है।
Tradução (pt): A vida é como uma caixa de chocolate.


---
## **Sumarização Automática**
Surge como resposta ao excesso de informação em contraste com o pouco tempo disponível para leitura. Destacam-se os seguintes pontos:

- Sumários extrativos (seleção de sentenças do texto original) vs. abstrativos (reescrevem o conteúdo);

- Mono vs. multidocumento;

- Variação de idioma (mono/multilíngue).

#### **Processo de Sumarização**

Um ponto central é que **identificar conteúdo relevante** não é trivial: desde o início da área, o grande desafio foi decidir o que é "mais importante" no texto-fonte. Com o tempo surgem desafios adicionais como sumarização multilíngue e abstrativa.

Algoritmo baseado em frequência:
- Conta as palavras do texto, calcula frequência relativa e usa isso para pontuar sentenças.
- Escolhe as sentenças com maior pontuação para compor o sumário.
- Destaca-se pela simplicidade e por ser um bom ponto de partida didático.

Algoritmo baseado em BERT:
- Usa um modelo de linguagem do tipo Transformer (BERT) para obter representações mais ricas do texto.
- Mostra uma nova etapa no processamento textual: redes neurais profundas + uso de GPU para lidar com SA.

#### **Avaliação em Sumarização Automática**

Informatividade
- Quanto da informação relevante do texto-fonte foi preservada no sumário;
- A métrica mais usada é a **ROUGE** que utiliza a comparação de resumos feitos por humanos e o obtido pelo algoritmo.

Qualidade Linguística
- Como ainda não há métricas automáticas robustas para aspectos como coesão, coerência e gramaticalidade, usa-se avaliação humana.
- Utiliza-se DUC (Document Understanding Conference) que define critérios a serem pontuados:
  - Gramaticalidade;
  - Não redundância;
  - Clareza referencial;
  - Foco;
  - Estrutura e coerência.

#### **Sumarização Abstrativa: Abordagem Google Pegasus**
Para que o modelo funcione adequadamente precisamos treiná-lo, mas para isso precisamos de muitos dados, são eles os **textos** e seus respectivos **resumos**.

É possível criar um ranking das sentenças mais importantes do texto. Assim, é escolhida a sentença com maior ranking (muito similar a sumarização extrativa), e o grande diferencial é que ao coletar a sentença mais importante removemos a mesma do texto original. Assim, tal sentença representaria o resumo abstrativo gerado pelo algoritmo. Assim, o mesmo irá "aprender" a parafrasear e gerar sumarização abstrativa.

In [ ]:
def sumarizar_tfidf(texto, num_sentencas=3):
    sentencas = re.split(r'(?<=[.!?])\s+', texto.strip())
    sentencas = [s for s in sentencas if s]
    if len(sentencas) <= num_sentencas:
        return texto

    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(sentencas)

    scores = X.sum(axis=1).A1

    idx_top = np.argsort(scores)[::-1][:num_sentencas]
    idx_top = sorted(idx_top)

    resumo = " ".join(sentencas[i] for i in idx_top)
    return resumo

texto_exemplo = """
A sumarização automática busca reduzir o tamanho de um texto preservando suas ideias principais.
Uma forma simples de fazer isso é selecionar as sentenças mais importantes.
Podemos usar TF-IDF para medir a relevância das palavras em cada sentença.
Depois, escolhemos as sentenças com maior pontuação e as unimos em um resumo.
Esse tipo de sumarização é chamada de extrativa, pois reutiliza sentenças do próprio texto.
"""

resumo = sumarizar_tfidf(texto_exemplo, num_sentencas=2)
print("TEXTO ORIGINAL:\n", texto_exemplo)
print("\nRESUMO GERADO:\n", resumo)


TEXTO ORIGINAL:
 
A sumarização automática busca reduzir o tamanho de um texto preservando suas ideias principais.
Uma forma simples de fazer isso é selecionar as sentenças mais importantes.
Podemos usar TF-IDF para medir a relevância das palavras em cada sentença.
Depois, escolhemos as sentenças com maior pontuação e as unimos em um resumo.
Esse tipo de sumarização é chamada de extrativa, pois reutiliza sentenças do próprio texto.


RESUMO GERADO:
 A sumarização automática busca reduzir o tamanho de um texto preservando suas ideias principais. Podemos usar TF-IDF para medir a relevância das palavras em cada sentença.


---
## **Detecção Automática de notícias falsas**

Fake news não é fenômeno recente: historicamente aparecia em fofocas, tabloides e mesmo em jornais tradicionais, com casos antigos de documentos forjados e reportagens falsas em eleições e desastres.

A diferença hoje é o alcance das redes sociais e apps de mensagem, que permitem disseminação massiva em qualquer país.

Eventos como eleições e a pandemia de COVID-19 evidenciam o impacto da desinformação; a própria IA gera, como efeito colateral, novos tipos de conteúdo enganoso.

#### **Conceitos e Contexto**

- **Deception**: Conteúdo enganoso, inclui notícias falsas, mas também hoaxes, sátira/ironia, reviews falsas e clickbaits.
- **Misinformation**: falso sem intenção de causar dano (erros, legendas erradas, estatísticas equivocadas).
- **Disinformation**: falso com intenção de prejudicar alguém ou algum grupo; é onde se encaixam, em geral, as fake news.
- **Malinformation**: informação verdadeira usada para causar dano (vazamento de dados, mudança de contexto, discurso de ódio).
- **Meias-verdades** (mistura de fatos verdadeiros e falsos) e pós-verdades, em que emoções e crenças pesam mais que fatos objetivos.

#### **Combate manual e dificuldade humana**
- Falta de conhecimento prévio sobre o tema;
- Viés de confirmação (as pessoas tendem a acreditar no que reforça suas crenças);
- Confiança excessiva em contatos próximos e pouca verificação antes de compartilhar;
- Falta de tempo/interesse para seguir recomendações de checagem.

#### **Métodos de detecção automática**
Aprendizado de máquina com atributos linguísticos:
- Medidas de inteligibilidade: legibilidade, coesão, diversidade lexical, complexidade sintática, organização discursiva, variáveis psicolinguísticas;
- Sentimento e subjetividade: Notícias falsas podem apresentar linguagem mais subjetiva/emotiva;
- Modelos discursivos: identifica padrões típicos de conteúdo enganoso.

Modelos de linguagem e deep learning:
- Potencial e desafios de usar LLMs na detecção de fake news, tanto para classificar quanto para gerar conteúdo enganoso.

Checagem automática de conteúdo:
- Extraem afirmações do texto;
- Buscam evidências em bases confiáveis;
- Comparam o conteúdo da notícia com essas evidências para indicar se é consistente ou conflita com os fatos.
- Pode ser utilizado RI (Recuperação de informação)

Credibilidade de fontes, sites e usuários:
- Análise de fontes de notícia;
- Análise de usuários e disseminadores.

In [ ]:

from transformers import pipeline

MODEL_NAME = "XSY/albert-base-v2-fakenews-discriminator"

fake_news_pipe = pipeline(
    task="text-classification",
    model=MODEL_NAME
)

textos = [
    "Scientists discover a new planet that could support human life.",
    "Breaking: Government announces that drinking coffee cures all diseases.",
    "Company X reports record profits in the last quarter.",
    "Social media post claims that the Earth has stopped rotating."
]

print("=== Detecção automática de fake news ===\n")
print("LABEL_0 → Fake news\nLABEL_1 → Notícia verdadeira\n")
for t in textos:
    result = fake_news_pipe(t)[0]
    label = result["label"]
    score = result["score"]
    print(f"Texto: {t}")
    print(f"→ Predição: {label} (confiança = {score:.3f})\n")


Device set to use cpu


=== Detecção automática de fake news ===

LABEL_0 → Fake news
LABEL_1 → Notícia verdadeira

Texto: Scientists discover a new planet that could support human life.
→ Predição: LABEL_1 (confiança = 0.933)

Texto: Breaking: Government announces that drinking coffee cures all diseases.
→ Predição: LABEL_0 (confiança = 1.000)

Texto: Company X reports record profits in the last quarter.
→ Predição: LABEL_1 (confiança = 0.994)

Texto: Social media post claims that the Earth has stopped rotating.
→ Predição: LABEL_1 (confiança = 0.639)



---
## **PLN e Humanidades Digitais**
Com o avanço do PLN surge a necessidade de ferramentas digitais de armazenamento, acesso e processamento de grandes volumes de dados, permitindo novos tipos de análise em pesquisas de humanidades.

#### **Preparação das fontes**
- Digitalização de documentos: OCR (reconhecimento óptico de caracteres);
- Metadados: acervo, identificador único, autoria, data, local, estrutura (volume, capítulo, página);
- Normalização: Lida com variações ortográficas antigas, abreviações, emojis, hashtags, etc;
- Anotação de corpora: Adiciona informações linguísticas (POS, sintaxe, entidades, eventos) representadas como metadados.

#### **Transformação de textos em dados**
- Extração de informação: Transformar dados não estruturados em estruturados;
- Representação de conhecimento e ontologias: Formalizar conceitos e relações em um formato processável por máquina.
- Seguir padrões de disponibilização e estruturação de dados.

---
## **PLN em Redes Sociais**
Redes sociais geram muito texto informal: gírias, abreviações, emojis, ironia etc., o que torna o processamento mais difícil que em textos formais.

Esses textos são valiosos como corpora reais para treinar e avaliar modelos de PLN, especialmente porque o Brasil é um dos países com maior presença nessas plataformas e o português domina essas interações.

#### **Principais áreas de aplicação em PLN para redes sociais**
Detecção de discurso de ódio e linguagem ofensiva
- Desafios importantes:
  - Escassez de dados anotados de qualidade em português.
  - Variação de formas de ódio (explícitas x veladas, interseccionalidade de ataques).
  - Necessidade de modelos que lidem com ironia e sarcasmo usados para mascarar ódio.

Análise de Sentimento
- Redes sociais e plataformas de avaliação (TripAdvisor, Booking, Airbnb, e-commerce etc.) geram muitos textos opinativos, úteis para tomada de decisão (consumidores, empresas, governo).

Detecção de notícias falsas

Detecção de ironia/sarcasmo/humor
- Busca-se:
  - Identificar padrões linguísticos,
  - Considerar contexto (hashtags, emojis, histórico de postagens),
  - Treinar modelos específicos para capturar essas nuances, ajudando inclusive a desmascarar discursos de ódio “camuflados”.

---
## **Questões Éticas em IA e PLN**
Algoritmos de aprendizado de máquina aprendem a partir de grandes conjuntos de dados. Se esses dados forem desbalanceados ou mal coletados, os modelos aprendem vieses e passam a se comportar de forma injusta ou inadequada, podendo ferir a ética.

Exemplo de viés:
- Reconhecimento facial que não identificava o rosto de Joy Buolamwini, uma mulher negra, a não ser quando ela usou uma máscara branca, porque 79,6% dos rostos do conjunto de treino eram de pessoas de pele clara.

#### **LGPD e IA**
Devemos nos atentar ao coletar dados para treinar modelos se não estamos ferindo alguma restrição do site ou coletando dados sensíveis.

Mesmo quando os dados estão publicamente acessíveis, não está claro se as pessoas sabiam que suas postagens seriam usadas para treinar modelos. Deve-se respeitar a propriedade intelectual e dos direitos autorais.

#### **Princípios para IA ética**

- Justiça, diversidade e não discriminação;
- Transparência e explicabilidade;
- Robustez técnica e segurança;
- Privacidade e proteção de dados;
- Responsabilidade e prestação de contas.

#### **Ética em PLN**
- Nos LLMs (Large Language Models), como o ChatGPT, a competência linguística vem de corpora gigantes coletados na internet. Só que o acesso à internet é desigual, então os dados de treino tendem a ser não representativos e a deixar de fora muitas variedades culturais e linguísticas.
- Como a internet está cheia de discurso tóxico, preconceitos, discriminação e posturas antiéticas, esses conteúdos podem ser aprendidos e depois reproduzidos nas saídas dos modelos, perpetuando e amplificando esses vieses.


#### **Modelos de língua como fonte de conhecimento?**
Modelos de língua impressionam por gerar textos coerentes e "bonitos", mas:

- Não "entendem" o que dizem: Apenas geram sequências de palavras estatisticamente prováveis a partir dos dados com que foram treinados;
- Não garantem autenticidade nem veracidade;
- Não é um "gerador de conhecimento".

---
#### **Referências**
* **[Processamento de Linguagem Natural: Conceitos, Técnicas e Aplicações em Português (3ª Ed.)](https://brasileiraspln.com/livro-pln/3a-edicao/)**
* **[Hugging Face](https://huggingface.co/)**

---

#### **Acompanhe a LADATA**  
[![Instagram](https://img.shields.io/badge/Instagram-%40ladata.ufs-E4405F?style=for-the-badge&logo=instagram&logoColor=white)](https://www.instagram.com/ladata.ufs/)  
[![GitHub](https://img.shields.io/badge/GitHub-ladata--ufs-100000?style=for-the-badge&logo=github&logoColor=white)](https://github.com/ladata-ufs)  
[![LinkedIn](https://img.shields.io/badge/LinkedIn-LADATA_UFS-0077B5?style=for-the-badge&logo=linkedin&logoColor=white)](https://br.linkedin.com/company/ladata-ufs)  

<small>
Processamento de Linguagem Natural: das abordagens clássicas aos LLMs | XI SEMAC - 2025 <br>
<strong>Liga Acadêmica de Ciência de Dados | LADATA</strong>
</small>